### Fasting time & Weight: Personal records from August 2019

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

# Load your CSV data into a Pandas DataFrame
df = pd.read_csv('/home/turo/Downloads/data.csv', names=['time_delta', 'date_added', 'weight'])

In [538]:
df.tail()

,time_delta,date_added,weight
1513,NaN,NaN,NaN
1514,NaN,10/20/2023,78.50
1515,NaN,NaN,NaN
1516,NaN,10/25/2023,78.85
1517,NaN,10/31/2023,79.80


### Handling & Conversion [time_delta] column

In [539]:
# Get rid of this 12/30/1899 15:25:00	
df = df.drop(index=1)
df = df.reset_index(drop=True)
df.head(4)

,time_delta,date_added,weight
0,NaN,7/30/2019 20:25:00,77.0
1,NaN,7/30/2019 20:25:00,NaN
2,24:35:00,NaN,NaN
3,NaN,NaN,NaN


In [540]:
#### some error values from google sheets
print (df[(df['time_delta'] == '#VALUE!') | (df['time_delta'] == '#ERROR!')] )

#### remove those 
df = df[~df['time_delta'].isin(['#ERROR!', '#VALUE!'])]

print ('********************************')

print (df[(df['time_delta'] == '#VALUE!') | (df['time_delta'] == '#ERROR!')] )


     time_delta          date_added  weight
1232    #ERROR!  11/21/2022 8:40 PM     NaN
1237    #VALUE!  11/26/2022 7:17 PM     NaN
********************************
Empty DataFrame
Columns: [time_delta, date_added, weight]
Index: []


In [541]:
# Fasting data was calculated not always - 24.00 means there was no calculation 
count_24_hours = (df['time_delta'] == '24:00:00').sum()
print("Count of '24:00:00' entries in time_delta column:", count_24_hours)

print ('********************************')

df[df.time_delta == '24:00:00']

Count of '24:00:00' entries in time_delta column: 338
********************************


,time_delta,date_added,weight
8,24:00:00,NaN,NaN
24,24:00:00,8/22/2019,NaN
25,24:00:00,8/23/2019 17:00,NaN
31,24:00:00,8/29/2019 11:58 PM,NaN
33,24:00:00,8/31/2019 4:20 PM,NaN
...,...,...,...
1506,24:00:00,8/22/2023,NaN
1507,24:00:00,8/23/2023,NaN
1508,24:00:00,8/24/2023,NaN
1509,24:00:00,8/25/2023,NaN


In [542]:
## replace 24:00 with 0
df['time_delta'] = df['time_delta'].replace('24:00:00', '0')
df[df.time_delta == '24:00:00']

,time_delta,date_added,weight


In [543]:
df.sample(n=10, random_state=42)

,time_delta,date_added,weight
51,17:10:00,9/18/2019 15:50,NaN
168,3:10:00,1/13/2020 11:10 PM,NaN
1470,0,7/17/2023,NaN
926,27:44:00,2/10/2022 9:31 PM,NaN
422,18:21:00,9/23/2020 11:19 PM,NaN
1090,15:31:00,7/11/2022 7:54 PM,NaN
585,17:56:00,3/5/2021 9:14 PM,NaN
220,19:30:00,3/5/2020 9:15 PM,NaN
1469,0,7/16/2023,NaN
1442,0,6/19/2023,NaN


In [544]:
df.isna().sum()

time_delta     108
date_added      28
weight        1424
dtype: int64

In [545]:
# NaN values in time_delta cause trouble during export to POSTGRES
df['time_delta'] = df['time_delta'].fillna(0)
df.isna().sum()

time_delta       0
date_added      28
weight        1424
dtype: int64

In [546]:
############# Main Conversion of [time_delta]

def convert_to_timedelta(value):
    if isinstance(value, str) and ':' in value:
        return pd.to_timedelta(value)
    elif isinstance(value, str) and value.isdigit():
        if value != '0':
            print(f"Found single number value: {value}")
        return pd.to_timedelta(f"{int(value)} hours")
    else:
        return pd.to_timedelta(value)

df['time_delta'].apply(convert_to_timedelta)


Found single number value: 18
Found single number value: 12
Found single number value: 16
Found single number value: 18
Found single number value: 41


0      0 days 00:00:00
1      0 days 00:00:00
2      1 days 00:35:00
3      0 days 00:00:00
4      0 days 00:00:00
             ...      
1512   0 days 00:00:00
1513   0 days 00:00:00
1514   0 days 00:00:00
1515   0 days 00:00:00
1516   0 days 00:00:00
Name: time_delta, Length: 1515, dtype: timedelta64[ns]

In [547]:
#### Drop these two digit values
#### We don't know if they are hours or minutes

# Define the values to be filtered
values_to_filter = ["18", "12", "16", "41"]

# Filter and print the rows where 'time_delta' contains any of the specified values
filtered_rows = df[df['time_delta'].isin(values_to_filter)]
print("Rows with specified values:")
print(filtered_rows)

# Drop the filtered rows from the DataFrame
df = df[~df['time_delta'].isin(values_to_filter)]
print ('*********************')

print (df[df['date_added'].str.contains('/11/2020 11:00 PM', na=False)] )

Rows with specified values:
    time_delta          date_added  weight
345         18    7/8/2020 9:56 PM     NaN
348         12  7/11/2020 11:00 PM     NaN
620         16    4/9/2021 9:28 PM     NaN
728         18   7/26/2021 7:58 PM     NaN
882         41  12/27/2021 5:49 PM     NaN
*********************
Empty DataFrame
Columns: [time_delta, date_added, weight]
Index: []


In [548]:
df['time_delta'] = df['time_delta'].apply(convert_to_timedelta)

In [549]:
## [time_delta] needs to be in seconds else error duirng export
df['time_delta'] = df['time_delta'].dt.total_seconds().astype("Int64")

### Handling & Conversion of ['date_added'] column

In [550]:
print (df[df['date_added'].str.contains('11/30/2020', na=False)] )

print ('********************************')

## Get Rid of that row
df = df[~df['date_added'].str.contains("11/30/2020", na=False)]

print (df[df['date_added'].str.contains('11/30/2020', na=False)] )

      time_delta          date_added  weight
490  -3815784660  11/30/2020 7:32 PM     NaN
********************************
Empty DataFrame
Columns: [time_delta, date_added, weight]
Index: []


In [551]:
# Dirty Manual Data Input  - 10:01pm - pm needs space

print (df[df['date_added'].str.contains('pm', na=False)] )


print ('********************************')

df['date_added'] = df['date_added'].replace('3/1/2021 17:36:00 10:01pm', '3/1/2021 17:36:00')

print (df[df['date_added'].str.contains('17:36:00', na=False)] )



     time_delta                 date_added  weight
581       56880  3/1/2021 17:36:00 10:01pm     NaN
********************************
     time_delta         date_added  weight
581       56880  3/1/2021 17:36:00     NaN


In [552]:
## dirty input in google sheets time is 23.56

print (df[df['date_added'].str.contains('23.56', na=False)] )

print ('*******************************')

# Find all lines with time and minutes separated by .
pattern = r".*\..*"
matched_rows = df[df['date_added'].str.contains(pattern, na=False)]
print(matched_rows)

print ('*******************************')

df['date_added'] = df['date_added'].replace('10/1/2021 23.56', '10/1/2021 23:56')

print (df[df['date_added'].str.contains('23:56', na=False)] )



     time_delta       date_added  weight
795       66420  10/1/2021 23.56    80.5
*******************************
     time_delta       date_added  weight
795       66420  10/1/2021 23.56    80.5
*******************************
     time_delta       date_added  weight
795       66420  10/1/2021 23:56    80.5


In [553]:
## date_added NaN causes date_added to be assigned to a current date (2024)

print (df[df['date_added'].isna()] )

print ('*******************************')

print (df['date_added'].isna().sum() )

      time_delta date_added  weight
2          88500        NaN     NaN
3              0        NaN     NaN
8              0        NaN     NaN
993            0        NaN     NaN
1001           0        NaN     NaN
1009           0        NaN     NaN
1017           0        NaN     NaN
1025           0        NaN     NaN
1033           0        NaN     NaN
1041           0        NaN     NaN
1049           0        NaN     NaN
1057           0        NaN     NaN
1065           0        NaN     NaN
1073           0        NaN     NaN
1081           0        NaN     NaN
1089           0        NaN     NaN
1097           0        NaN     NaN
1105           0        NaN     NaN
1113           0        NaN     NaN
1121           0        NaN     NaN
1129           0        NaN     NaN
1200           0        NaN     NaN
1208           0        NaN     NaN
1216           0        NaN     NaN
1224           0        NaN     NaN
1511           0        NaN     NaN
1512           0        NaN 

In [554]:
df.shape

(1509, 3)

In [555]:
df = df.dropna(subset=['date_added'])

In [556]:
df.shape

(1481, 3)

In [557]:
###################### Main Conversion 
# Define a mask to filter rows with the specified format ( 01/01/2022 12:34:56 )
mask = df['date_added'].str.match(r'\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2}:\d{2}')

# Convert only the rows that match the specified format
df.loc[mask, 'date_added'] = pd.to_datetime(df.loc[mask, 'date_added'], errors='coerce', format='%m/%d/%Y %H:%M:%S')

# Convert the remaining date strings to datetime - e.g. 9/14/2019 4:00 PM
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Format the datetime values as required for PostgreSQL
df['date_added'] = df['date_added'].dt.strftime('%Y-%m-%d %H:%M:%S')


#### Handling of ['weight] column

In [558]:
# Replace NaN values with None for weight column
df['weight'] = df['weight'].where(~df['weight'].isna(), None)

In [559]:
df.head()

,time_delta,date_added,weight
0,0,2019-07-30 20:25:00,77.0
1,0,2019-07-30 20:25:00,NaN
4,0,2019-02-08 21:25:00,NaN
5,0,2019-03-08 15:45:00,NaN
6,0,2019-04-08 22:20:00,NaN


In [560]:
df.tail()

,time_delta,date_added,weight
1509,0,2023-08-25 00:00:00,NaN
1510,0,2023-08-26 00:00:00,NaN
1513,0,2023-10-20 00:00:00,78.50
1515,0,2023-10-25 00:00:00,78.85
1516,0,2023-10-31 00:00:00,79.80


In [561]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1481 entries, 0 to 1516
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   time_delta  1481 non-null   Int64  
 1   date_added  1481 non-null   object 
 2   weight      91 non-null     float64
dtypes: Int64(1), float64(1), object(1)
memory usage: 47.7+ KB


In [562]:
### Filter for time_delta 0 and Weight NaN
df[(df['time_delta'].isin([0])) & (df['weight'].isna())]


,time_delta,date_added,weight
1,0,2019-07-30 20:25:00,NaN
4,0,2019-02-08 21:25:00,NaN
5,0,2019-03-08 15:45:00,NaN
6,0,2019-04-08 22:20:00,NaN
7,0,2019-05-08 14:15:00,NaN
...,...,...,...
1506,0,2023-08-22 00:00:00,NaN
1507,0,2023-08-23 00:00:00,NaN
1508,0,2023-08-24 00:00:00,NaN
1509,0,2023-08-25 00:00:00,NaN


In [563]:
df.shape

(1481, 3)

In [564]:
rows_to_remove = df[(df['time_delta'].isin([0])) & (df['weight'].isna())]
df = df.drop(rows_to_remove.index)

In [565]:
df.shape

(1093, 3)

In [566]:
df

,time_delta,date_added,weight
0,0,2019-07-30 20:25:00,77.00
10,61620,2019-08-08 00:00:00,77.40
13,46800,2019-08-11 02:10:00,NaN
14,78600,2019-08-12 10:00:00,NaN
15,50400,2019-08-13 12:36:00,NaN
...,...,...,...
1461,0,2023-07-08 00:00:00,78.27
1479,0,2023-07-26 00:00:00,78.56
1513,0,2023-10-20 00:00:00,78.50
1515,0,2023-10-25 00:00:00,78.85


### Export Data

In [567]:
############# We just check connection with DB here 

import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()  # Load environment variables from .env file

DATABASE_URL = os.environ.get('DATABASE_URL')
engine = create_engine(DATABASE_URL)

# Check if the connection is valid
if engine.connect():
    print("Connection successful!")
else:
    print("Connection failed!")

from sqlalchemy import text

# Establish a connection
with engine.connect() as connection:
    query = text("SELECT * FROM eat LIMIT 3")
    results = connection.execute(query).fetchall()

for row in results:
    print(row)

Connection successful!


In [568]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from app.models import eat  # Import your eat model
from datetime import datetime, timedelta

import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

DATABASE_URL = os.environ.get('DATABASE_URL')
engine = create_engine(DATABASE_URL)

Session = sessionmaker(bind=engine)
session = Session()

for index, row in df.iterrows():
    # Convert 'time_delta' to a valid interval value (0 seconds)
    time_delta = timedelta(seconds=int(row['time_delta']))
    
    # Convert 'date_added' to a valid timestamp
    date_added = None
    if pd.notna(row['date_added']):
        date_added = datetime.strptime(row['date_added'], '%Y-%m-%d %H:%M:%S')
    
    # Export the row to the eat table
    eat_instance = eat(
        meal=None,  # Adjust as needed
        comment=None,  # Adjust as needed
        time_delta=time_delta,
        date_added=date_added,
        weight=row['weight']
    )
    session.add(eat_instance)

# Commit changes to the database
session.commit()


In [569]:
df.shape

(1093, 3)

In [4]:
import os

# Get the current working directory
current_working_directory = os.getcwd()

# Print the current working directory
print(f"Current working directory: {current_working_directory}")


Current working directory: /home/turo/projects/life_quantifed


In [5]:
df.to_csv('data/cleaned_data.csv', index=False)
